Funcoes utilizadas
Criar uma lista de comprimentos de strings em cada célula da coluna especificada
comprimentos = [len(str(celula)) for celula in df.iloc[: ,4]]
Encontrar o índice da célula com o maior número de caracteres
indice_maior = comprimentos.index(max(comprimentos))
Extrair a célula com o maior número de caracteres
celula_maior = df.iloc[indice_maior][4] df.iloc[: , 4][indice_maior] df.iloc[: , 4].value_counts() celula_maior df.iloc[:,0]
index = df.columns.get_loc(‘abcz’)
index df

IDANIMAL = CHAVE UNICA
MP = CHAVE UNICA -  Mes parto
AP = CHAVE UNICA - -Ano parto

LTOT = lactacao total
L365 = lactacao em ate 365 dias
L305 = lactacao em ate 305 dias
DE = Data de Secagem
ME = Mes de secagem
AE = ANO de Secagem
caso o LTOT, L365 e L305, DE, ME e AE sejam iguais ao arquivo old, pode ser descartado

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## conectando no banco de dados

In [3]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="4d5f0l9k",
  database="bioinformatica_new"
)

## Retornando toda tabela genealogia_embrapa e genealogia_abcz e tabela criadores do banco de dados para dataframes

In [4]:
sql = f"WITH tabela_virtual AS (SELECT * FROM genealogia_embrapa) SELECT tv.*, (SELECT registro FROM tabela_virtual AS pai WHERE pai.id_produto = tv.id_touro) AS registro_pai, (SELECT registro FROM tabela_virtual AS mae WHERE mae.id_produto = tv.id_vaca) AS registro_mae FROM tabela_virtual AS tv"
sql1 = "SELECT * FROM genealogia_abcz"
sql2 = "SELECT * FROM criadores"
sql3 = "SELECT * FROM producao_embrapa"
df_genealogia_embrapa = pd.read_sql(sql, mydb)
df_genealogia_embrapa=df_genealogia_embrapa.rename(columns={'idanimal': 'IDANIMAL'})
df_genealogia_abcz = pd.read_sql(sql1, mydb)
df_criadores = pd.read_sql(sql2, mydb)
df_producao = pd.read_sql(sql3, mydb)


## Lendo planilhas

In [5]:
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')
dados_2023 = pd.read_csv("PNMGL - Dados girolando - 17-02-23.txt", encoding='ISO-8859-1')
dados_2022 = pd.read_csv("PNMGL - Dados_old - 17-02-2023.txt", encoding='ISO-8859-1')

In [5]:
pedigree

,NOMEVACA,RGVACA,COMPRAC,IDANIMAL,DNASC,SEXO,NOMEPAI,RGPAI,GSPAI,IDPAI,NOMEMAE,RGMAE,GSMAE,IDMAE,gs,cgen_a,comb
1,NaN,RF-0618,11,616612.0,0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,GIR,11
2,NaN,RF-1285,11,616611.0,0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,GIR,11
3,4527 DE IRAI,AJ-4527,34,766580.0,20/01/2006,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,3/4HOL 1/4GIR,795
4,4530 DE IRAI,AJ-4530,34,766583.0,03/11/2004,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,3/4HOL 1/4GIR,795
5,C551 OSKA,7932-BV,12,3410926.0,08/11/2020,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1/2HOL 1/2GIR,519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276760,ZUZU ZU,0050,34,672063.0,29/03/2001,F,MIKLIN GALAXY-ET,AX102336,10.0,4653.0,AMORA Z.U.,M-8808,12.0,155290.0,6,3/4HOL 1/4GIR,795
2276761,ZUZUCA FABIANO,L-5947,34,335253.0,04/05/1995,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,3/4HOL 1/4GIR,795
2276762,ZUZUCA FIV KENYO,0202-BI,12,2358145.0,18/02/2019,F,BACON-HILL MONTROSS-ET,AX145128,10.0,1603021.0,KATARINA FIV KENYO,KOK690,11.0,1995568.0,4,1/2HOL 1/2GIR,519
2276763,ZYCH FABIANA 18,BR1235804,10,3114806.0,0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,HOL,999


In [6]:
dados_2022

,ordem,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,...,QTD_ORDENHAS,nasc,parto,secagem,anopc,anop_menos1c,rebc,rac,nordc,rebanoe
0,1,2381,TULIPA MODELO JM NOVO HORIZONT,14,3670-U,MODELO TE DE BRASILIA,11,B5213,FABULA SEGREDO JM NOVO HORIZON,12,...,2,2011-07-13,2018-04-26,2018-11-23,2018,2017,2381,2,2,23812018222
1,2,3093,ALTEROSA SM FIV MITU,12,9827-A,MORNINGVIEW MISCHIEF-ET,10,AX125529,CAIANA FIV KENYO,11,...,2,2009-07-13,2019-03-15,2020-06-22,2019,2018,3093,2,2,30932018122
2,3,2384,RODOVIA 6013 FAMOSO JM MONTE A,38,1094-T,FAMOSO TE SILVANIA,11,EFC779,ATICA PARAMOUNT FIV JM MONTE A,34,...,2,2013-03-18,2019-03-26,2020-01-10,2019,2018,2384,2,2,23842018122
3,4,7453,ARAPONGA 803 FA DA SERRA,34,8590-AB,PALMCREST MONUMENT-ET,10,AX127702,ADRIANA FA DA SERRA,12,...,2,2013-10-21,2017-10-01,2018-12-20,2017,2016,7453,2,2,74532017122
4,5,5569,FLORA TEATRO DO SEGREDO VELHO,14,6986-J,TEATRO DA SILVANIA,11,EFC383,LUA DO SEGREDO VELHO,12,...,2,2012-07-20,2020-02-01,2020-06-12,2020,2019,5569,2,2,55692019122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,5206,9734,TIEMI CONDESSA ELUDE FIV,12,1186-AR,SILVERRIDGE V ELUDE,10,AX145140,COLONIA TE AG,11,...,2,2016-07-10,2020-08-21,2020-12-29,2020,2019,9734,2,2,97342020222
5206,5207,9257,GUISA FIV MCCUTCHEN FBE,12,8181-AS,DE-SU BKM MCCUTCHEN 1174-ET,10,AX140261,MADAME ISIS 2B,11,...,2,2016-08-22,2020-05-25,2021-02-24,2020,2019,9257,3,2,92572020232
5207,5208,9692,DAGUA MOGUL FIV MOURA LEITE,34,4050-AV,MOUNTFIELD SSI DCY MOGUL-ET,10,AX138953,QUITANDA FIV PLANET VR,12,...,2,2017-06-22,2020-12-20,2021-12-05,2020,2019,9692,2,2,96922020122
5208,5209,5192,KARINA FIV BEEMER 2244 RS DO R,34,5851-AX,POL BUTTE MC BEEMER-ET,10,AX145127,ENGENHO DA RAINHA BIANCA,12,...,3,2018-02-04,2020-04-01,2020-08-06,2020,2019,5192,3,3,51922020233


In [7]:
dados_2023

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,QTD_ORDENHAS,nasc,parto,secagem,anopc,anop_menos1c,rebc,rac,nordc,rebanoe
139,3093,ALTEROSA SM FIV MITU,12,9827-A,MORNINGVIEW MISCHIEF-ET,10,AX125529,CAIANA FIV KENYO,11,KOK31,...,2,2009-07-13,2019-03-15,2020-06-22,2019,2018,3093,2,2,30932018122
162,9257,ESCOCIA FIV GABINETE DO R.DA P,12,8494-BA,GABINETE SILVANIA,11,EFC946,BRAUNAS SPIRTE ROSE KATRINA-TE,10,BX381125,...,2,2017-07-05,2019-12-20,2020-11-26,2019,2018,9257,2,2,92572019122
221,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,2015-03-11,2020-01-13,2021-11-11,2020,2019,7461,2,2,74612019122
222,5569,FLORA TEATRO DO SEGREDO VELHO,14,6986-J,TEATRO DA SILVANIA,11,EFC383,LUA DO SEGREDO VELHO,12,Y-5738,...,2,2012-07-20,2020-02-01,2020-06-12,2020,2019,5569,2,2,55692019122
250,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2014-12-22,2022-02-08,2022-10-15,2022,2021,10639,2,2,106392021122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391297,1129,BATALHA UFP ANGICO II,38,D-0636,DESEJO TE DA SILVANIA,11,EFC645,MIUDA ANGICO II,34,AJ-7860,...,2,2008-08-25,2019-07-25,2020-04-01,2019,2018,1129,2,2,11292019222
391442,11844,MONTANA 1238 TEATRO FIV JM MON,12,6848-AY,TEATRO DA SILVANIA,11,EFC383,547 JM MONTE ALVERNE,10,C-0434,...,2,2016-07-14,2019-03-04,2019-06-15,2019,2018,11844,2,2,118442018122
391489,6915,RAIZ FIV DAS ARABIAS II,14,3835-AD,C.A.SANSAO,11,KCA472,PALAFITA DAS ARABIAS,12,AD-0906,...,2,2011-10-29,2020-08-14,2020-10-31,2020,2019,6915,2,2,69152020222
391670,7461,NINA IMPERADOR BAXTER RF SAO F,5858,6352-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JORDANA MORRELL RF SAO FRANCIS,58,6872-N,...,2,2015-02-23,2019-03-16,2019-12-12,2019,2018,7461,2,2,74612018122


In [8]:
df_genealogia_embrapa

,id_produto,id,abcz,registro,nome,sexo,paternidade,ck_sire,id_touro,id_vaca,...,gr_tp,rebc2_cri,rebc2_pr,cat,ficha_a,cod_prod,IDANIMAL,origem,registro_pai,registro_mae
0,0001,0001,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
1,0003,0003,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
2,0006,0006,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
3,02418-249,02418-249,None,2418,nan,F,None,None,108115,108615,...,0,99999,99999,None,0,0,0,OUTRAS,A1343,18935
4,03,03,None,None,nan,M,None,None,None,None,...,0,99999,99999,None,0,0,0,OUTRAS,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463543,ZZZ7,ZZZ7,S,ZZZ7,AMBICAO,F,None,None,KCA472,135177,...,0,7145-ABCZ 1,11958-ABCZ 1,PO,1351780,3158020,0,ABCGIL,KCA472,KB429
463544,ZZZ8,ZZZ8,S,ZZZ8,AGASALHO,M,None,None,121118,131985,...,0,7145-ABCZ 1,7145-ABCZ 1,PO,0,3247323,0,ABCZ,K830,KA7368
463545,ZZZ9,ZZZ9,S,ZZZ9,BACABAL,M,None,None,133175,178922,...,0,7145-ABCZ 1,7145-ABCZ 1,PO,0,3247322,0,ABCZ,K974,KA9422
463546,ZZZA1,ZZZA1,S,ZZZA1,ANITA,F,None,None,EFC383,JDRB874,...,0,7145-ABCZ 1,7145-ABCZ 1,PO,0,9499094,0,ABCZ,EFC383,JDRB874


In [9]:
df_genealogia_abcz

,cod_prod,idGenealogiaABCZ,rgd,rgn,sexo,cat,cod_pai,cod_mae,nasc,cgen_a,...,codcri,fazcri,rebc_cri,rebc2_cri,codprop,fazprop,rebc_prop,rebc2_prop,rgd2,dt_inser
0,44,None,U3983,379,F,PO,861789,849556,1981-10-11,GIR,...,2761,2,2761-ABCZ,2761-ABCZ 2,7192,1,7192-ABCZ,7192-ABCZ 1,U3983,None
1,99,None,T2799,153,F,PO,1955144,848578,1977-06-05,GIR,...,494,1,494-ABCZ,494-ABCZ 1,7272,1,7272-ABCZ,7272-ABCZ 1,T2799,None
2,100,None,A7488,3755,M,PO,377108,1615593,1987-06-04,GIR,...,2851,1,2851-ABCZ,2851-ABCZ 1,7275,1,7275-ABCZ,7275-ABCZ 1,A7488,None
3,101,None,X6732,3792,F,PO,1615565,1615684,1987-08-04,GIR,...,2851,1,2851-ABCZ,2851-ABCZ 1,2955,1,2955-ABCZ,2955-ABCZ 1,X6732,None
4,102,None,X6733,3798,F,PO,1615802,1615704,1987-08-20,GIR,...,2851,1,2851-ABCZ,2851-ABCZ 1,7275,1,7275-ABCZ,7275-ABCZ 1,X6733,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745022,19650743,None,ABSA,21,F,PO,16900821,17558879,2022-07-21,GIR,...,12626,1,12626-ABCZ,12626-ABCZ 1,12626,1,12626-ABCZ,12626-ABCZ 1,ABSA21,None
745023,19652061,None,RRFZ,35,F,PO,9265878,15828344,2022-03-02,GIR,...,38042,1,38042-ABCZ,38042-ABCZ 1,38042,1,38042-ABCZ,38042-ABCZ 1,RRFZ35,None
745024,19652062,None,RRFZ,37,F,PO,9265878,15186511,2022-04-29,GIR,...,38042,1,38042-ABCZ,38042-ABCZ 1,38042,1,38042-ABCZ,38042-ABCZ 1,RRFZ37,None
745025,19652084,None,RRFZ,38,F,PO,9265878,17598634,2022-05-23,GIR,...,38042,1,38042-ABCZ,38042-ABCZ 1,38042,1,38042-ABCZ,38042-ABCZ 1,RRFZ38,None


In [10]:
df_criadores

,rebc_or,idCriador,nome_criador,codz,codgil,codgiro,fonte,data_fonte,reb
0,10-ABCZ,None,ADIR DO CARMO LEONEL,10,10010,0,nan,None,10-ABCZ
1,1001-GIRO,None,LEONARDO FAUSTINO MARQUES,0,0,1001,GIROLANDO202002,None,1001-GIRO
2,1002,None,EDUARDO F FERREIRA,0,1002,0,nan,None,1002
3,10022-ABCZ,None,PALESTINA AGROPAST LTDA,10022,0,0,ABCZ202003,None,10022-ABCZ
4,10024-ABCZ,None,ANTONIO CARLOS BARBOSA ALVARES,10024,0,0,ABCZ202003,None,10024-ABCZ
...,...,...,...,...,...,...,...,...,...
2367,9964-GIRO,None,nan,0,0,9964,GIRO20170301,None,9964-GIRO
2368,9965-ABCZ,None,PAOLA GAZZINELLI,9965,0,0,ABCZ202003,None,9965-ABCZ
2369,9988-GIRO,None,nan,0,0,9988,GIRO20170301,None,9988-GIRO
2370,9991-ABCZ,None,AGRO PASTORIL DOS POCOES LTDA,9991,272,0,nan,None,9991-ABCZ


In [11]:
df_producao

,chaveprod,fonte,prio_fonte,codgil,codz,faz,codgiro,rebc_or,rebc2_or,reb,...,psolid,psol305,ccs,porgor,porprot,porlac,porsol,torneio,pl305tor,nctor
0,02418-249_1975-05,2011,36,34962,1180,10,0,36234-ABCZ,36234-ABCZ 1,36234-ABCZ,...,0,0,0,0.00,0.0,0.0,0.0,None,0,0
1,1-249_1990-12,2011,36,34962,1180,10,0,36234-ABCZ,36234-ABCZ 1,36234-ABCZ,...,0,0,0,4.39,0.0,0.0,0.0,None,0,0
2,1-281_1988-09,2011,36,281,0,0,0,281,281,281,...,0,0,0,0.00,0.0,0.0,0.0,None,0,0
3,1-281_1990-05,2011,36,281,0,0,0,281,281,281,...,0,0,0,3.58,0.0,0.0,0.0,None,0,0
4,1-353_1996-03,2011,36,353,0,0,0,353,353,353,...,0,0,0,4.48,0.0,0.0,0.0,None,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176707,ZZZ14_2009-05,ABCZ202112_leit,5,37040,27040,10,0,27040-ABCZ,27040-ABCZ 1,27040-ABCZ,...,0,0,0,0.00,0.0,0.0,0.0,None,0,0
176708,ZZZ7_2005-11,ABCZ202112_gord,5,21958,11958,10,0,11958-ABCZ,11958-ABCZ 1,11958-ABCZ,...,0,0,0,5.47,0.0,0.0,0.0,None,0,0
176709,ZZZ7_2007-06,ABCZ202112_leit,5,21958,11958,10,0,11958-ABCZ,11958-ABCZ 1,11958-ABCZ,...,0,0,0,0.00,0.0,0.0,0.0,None,0,0
176710,ZZZ7_2009-04,ABCZ202112_leit,5,21958,11958,10,0,11958-ABCZ,11958-ABCZ 1,11958-ABCZ,...,0,0,0,0.00,0.0,0.0,0.0,None,0,0


## Criando um dataframe que é a junção dos dois dataframes, dados_2022 e dados_2023, pelas celulas que possuem os mesmos valores nas colunas IDANIMAL, AP, MP, L305, LTOT, L365, ME e AE.

In [6]:
df_dados_sem_atualizacao_entre_2022_2023 = pd.merge(dados_2023, dados_2022, on=['IDANIMAL', 'AP', 'MP','L305', 'LTOT', 'L365', 'ME', 'AE', 'CAUSA'],suffixes=('', '_drop'))
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop(df_dados_sem_atualizacao_entre_2022_2023.filter(regex='_drop$').columns, axis=1)
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop_duplicates()
df_dados_sem_atualizacao_entre_2022_2023

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nasc,parto,secagem,anopc,anop_menos1c,rebc,rac,nordc,rebanoe,ordem
0,3093,ALTEROSA SM FIV MITU,12,9827-A,MORNINGVIEW MISCHIEF-ET,10,AX125529,CAIANA FIV KENYO,11,KOK31,...,2009-07-13,2019-03-15,2020-06-22,2019,2018,3093,2,2,30932018122,2
1,9257,ESCOCIA FIV GABINETE DO R.DA P,12,8494-BA,GABINETE SILVANIA,11,EFC946,BRAUNAS SPIRTE ROSE KATRINA-TE,10,BX381125,...,2017-07-05,2019-12-20,2020-11-26,2019,2018,9257,2,2,92572019122,587
2,5569,FLORA TEATRO DO SEGREDO VELHO,14,6986-J,TEATRO DA SILVANIA,11,EFC383,LUA DO SEGREDO VELHO,12,Y-5738,...,2012-07-20,2020-02-01,2020-06-12,2020,2019,5569,2,2,55692019122,5
3,7789,CABALA BRIAR 613 FUNCHAL,12,8178-AD,OLSZEWSKI ALTABRIAR,10,AX137375,1760 FUNCHAL CURVELO,11,B-1760,...,2014-10-26,2018-11-23,2019-03-26,2018,2017,7789,2,2,77892018122,1352
4,8821,9057 WACLAW VALE OURO SILVANIA,12,4230-AI,VALE OURO TE SILVANIA,11,EFC464,AGRINDUS 3432 JUNEAU,10,BX412129,...,2015-08-10,2018-10-20,2019-08-10,2018,2017,8821,2,3,88212018123,5070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2828,6329,LARANJA SHOTTLE STT,12,2447-AX,MENGE SHOTTLE X-630-TE,10,AX132404,BIRIBA TE ZBR,11,ZBR24,...,2017-08-22,2020-08-14,2021-04-21,2020,2019,6329,2,2,63292020222,3010
2829,1129,BATALHA UFP ANGICO II,38,D-0636,DESEJO TE DA SILVANIA,11,EFC645,MIUDA ANGICO II,34,AJ-7860,...,2008-08-25,2019-07-25,2020-04-01,2019,2018,1129,2,2,11292019222,3580
2830,11844,MONTANA 1238 TEATRO FIV JM MON,12,6848-AY,TEATRO DA SILVANIA,11,EFC383,547 JM MONTE ALVERNE,10,C-0434,...,2016-07-14,2019-03-04,2019-06-15,2019,2018,11844,2,2,118442018122,3586
2831,6915,RAIZ FIV DAS ARABIAS II,14,3835-AD,C.A.SANSAO,11,KCA472,PALAFITA DAS ARABIAS,12,AD-0906,...,2011-10-29,2020-08-14,2020-10-31,2020,2019,6915,2,2,69152020222,3592


## Criando um dataframe com somente os novos dados inseridos em 2023
Esses são os dados que precisam ser inseridos no banco de dados

In [7]:
# faz o merge entre os dois dataframes, o dados com os dados repetidos
df_merge = pd.merge(dados_2023, df_dados_sem_atualizacao_entre_2022_2023, how='left', indicator=True, suffixes=('', '_drop'))

# filtra somente as linhas que estão no dataframe da esquerda
df_dados_novos_para_insercao_no_banco_de_dados = df_merge[df_merge['_merge'] == 'left_only']

# remove a coluna "_merge"
df_dados_novos_para_insercao_no_banco_de_dados.drop('_merge', axis=1, inplace=True)

df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_merge.filter(regex='_drop$').columns, axis=1)
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nasc,parto,secagem,anopc,anop_menos1c,rebc,rac,nordc,rebanoe,ordem
2,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2015-03-11,2020-01-13,2021-11-11,2020,2019,7461,2,2,74612019122,NaN
4,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2014-12-22,2022-02-08,2022-10-15,2022,2021,10639,2,2,106392021122,NaN
5,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2013-10-18,2021-02-18,2021-12-20,2021,2020,6039,2,2,60392020122,NaN
13,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2015-04-27,2021-06-16,2022-02-24,2021,2020,1945,2,2,19452021222,NaN
14,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2016-02-05,2021-07-13,2022-05-22,2021,2020,73119,2,2,731192021222,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3843,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2013-04-18,2021-04-22,2022-03-04,2021,2020,6029,2,2,60292021222,NaN
3851,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2015-04-25,2019-04-25,2019-09-14,2019,2018,132,2,2,1322019222,NaN
3853,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2019-06-13,2021-09-15,2022-06-10,2021,2020,6915,2,2,69152021222,NaN
3864,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2016-07-16,2019-12-10,2020-03-21,2019,2018,6029,2,2,60292019122,NaN


# Adicionando coluna GS, oriunda do dataframe Pedigree, no dataframe df_dados_novos_para_insercao_no_banco_de_dados
inclusão necessária para localizar o animal

In [8]:
df_temporaria = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, pedigree[['IDANIMAL', 'gs', 'SEXO']], on=['IDANIMAL'], suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria
df_dados_novos_para_insercao_no_banco_de_dados

nome_antigo = 'SEXO'
nome_novo = 'sexo'
indice_coluna = df_dados_novos_para_insercao_no_banco_de_dados.columns.get_loc(nome_antigo)
df_dados_novos_para_insercao_no_banco_de_dados.rename(columns={nome_antigo: nome_novo}, inplace=True)
df_dados_novos_para_insercao_no_banco_de_dados

#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.rename(columns={'SEXO': 'sexo'})

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,secagem,anopc,anop_menos1c,rebc,rac,nordc,rebanoe,ordem,gs,sexo
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2021-11-11,2020,2019,7461,2,2,74612019122,NaN,5,F
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2022-10-15,2022,2021,10639,2,2,106392021122,NaN,4,F
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2021-12-20,2021,2020,6039,2,2,60392020122,NaN,4,F
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2022-02-24,2021,2020,1945,2,2,19452021222,NaN,4,F
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2022-05-22,2021,2020,73119,2,2,731192021222,NaN,4,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2022-03-04,2021,2020,6029,2,2,60292021222,NaN,2,F
1035,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2019-09-14,2019,2018,132,2,2,1322019222,NaN,6,F
1036,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2022-06-10,2021,2020,6915,2,2,69152021222,NaN,5,F
1037,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2020-03-21,2019,2018,6029,2,2,60292019122,NaN,5,F


# Criando subtabela com somente as linhas que possuem gs = 0 no dataframe df_dados_novos_para_insercao_no_banco_de_dados

In [9]:
sub_tabela_gs_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['gs'] == 0]
sub_tabela_gs_0

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,secagem,anopc,anop_menos1c,rebc,rac,nordc,rebanoe,ordem,gs,sexo
53,5191,IBERICA FARDO,11,APKK18,FARDO FIV F.MUTUM,11,MUT697,ESPANHOLA,11,APKK1,...,2021-04-14,2020,2019,5191,2,2,51912020222,NaN,0,F
79,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,2022-08-25,2022,2021,1945,2,2,19452022222,NaN,0,F
81,4667,DAMA F.INDIANA,11,SNFP140,FARDO FIV F.MUTUM,11,MUT697,SOBERANA F. INDIANA,11,Z3236,...,2022-01-28,2021,2020,4667,2,2,46672021222,NaN,0,F
99,4667,MEIA NOITE F.INDIANA,11,SNFN48,MAESTRO TE F.MUTUM,11,MUT214,ESTRELA F.INDIANA,11,SNFP87,...,2022-07-06,2021,2020,4667,2,2,46672020122,NaN,0,F
184,5192,URUPIANGA FIV DE BRAS.,11,RRP8772,JAGUAR TE DO GAVIAO,11,GAV291,HONDA FIV DE BRAS.,11,RRP6268,...,2022-07-13,2021,2020,5192,3,2,51922021232,NaN,0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,5192,PILLAR FIV DE BRAS.,11,RRP8130,DIAMANTE TE DE BRASILIA,11,RRP5640,JAMARIA FIV DE BRASILIA,11,RRP6958,...,2022-11-12,2022,2021,5192,3,2,51922021132,NaN,0,F
995,4667,IRLANDA F. INDIANA,11,SNFN16,FARDO FIV F.MUTUM,11,MUT697,LONDRINA F. INDIANA,11,SNFP44,...,2022-10-08,2022,2021,4667,2,2,46672022222,NaN,0,F
1000,6915,DEJUA DAS ARABIAS,11,RHBR63,MAJOR TE DOS POCOES,11,APPG801,DEJUA TE F.MUTUM,11,MUT395,...,2022-04-26,2021,2020,6915,2,2,69152021222,NaN,0,F
1002,6915,PRATA DAS ARABIAS,11,RHBR49,VALE OURO TE SILVANIA,11,EFC464,JADE,11,RHBR4,...,2022-03-01,2021,2020,6915,2,2,69152021222,NaN,0,F


# Adicionando coluna cod_prod, oriunda do dataframe df_genealogia_abcz, na subtabela gs_0 dos animais que sao gs=0

In [10]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
df_temporaria1 = pd.merge(sub_tabela_gs_0, df_genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
sub_tabela_gs_0=df_temporaria1
sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()
sub_tabela_gs_0

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae
0,5191,IBERICA FARDO,11,APKK18,FARDO FIV F.MUTUM,11,MUT697,ESPANHOLA,11,APKK1,...,5191,2,2,51912020222,NaN,0,F,13672891.0,8530023.0,10614116.0
1,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,1945,2,2,19452022222,NaN,0,F,12005290.0,1615985.0,10091814.0
2,4667,DAMA F.INDIANA,11,SNFP140,FARDO FIV F.MUTUM,11,MUT697,SOBERANA F. INDIANA,11,Z3236,...,4667,2,2,46672021222,NaN,0,F,12175694.0,8530023.0,10905624.0
3,4667,MEIA NOITE F.INDIANA,11,SNFN48,MAESTRO TE F.MUTUM,11,MUT214,ESTRELA F.INDIANA,11,SNFP87,...,4667,2,2,46672020122,NaN,0,F,12971292.0,5932826.0,9691642.0
4,5192,URUPIANGA FIV DE BRAS.,11,RRP8772,JAGUAR TE DO GAVIAO,11,GAV291,HONDA FIV DE BRAS.,11,RRP6268,...,5192,3,2,51922021232,NaN,0,F,17054525.0,4176523.0,13961623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,5192,PILLAR FIV DE BRAS.,11,RRP8130,DIAMANTE TE DE BRASILIA,11,RRP5640,JAMARIA FIV DE BRASILIA,11,RRP6958,...,5192,3,2,51922021132,NaN,0,F,14820082.0,6716196.0,11600828.0
75,4667,IRLANDA F. INDIANA,11,SNFN16,FARDO FIV F.MUTUM,11,MUT697,LONDRINA F. INDIANA,11,SNFP44,...,4667,2,2,46672022222,NaN,0,F,11921127.0,8530023.0,6352036.0
76,6915,DEJUA DAS ARABIAS,11,RHBR63,MAJOR TE DOS POCOES,11,APPG801,DEJUA TE F.MUTUM,11,MUT395,...,6915,2,2,69152021222,NaN,0,F,13478123.0,3196251.0,8006397.0
77,6915,PRATA DAS ARABIAS,11,RHBR49,VALE OURO TE SILVANIA,11,EFC464,JADE,11,RHBR4,...,6915,2,2,69152021222,NaN,0,F,11470928.0,3890526.0,9469537.0


# Adicionar o cod_pai dos animais em que o pai tem o gspai = 11

In [13]:
sub_tabela_cod_pai_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['gs_pai'] == 0]
df_temporaria2 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'gs_pai']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
sub_tabela_cod_pai_0 = df_temporaria2
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
sub_tabela_cod_pai_0

KeyError: 'gs_pai'

# adicionar o cod_mae dos animais em que a mae tem o gsmae = 11

In [ ]:
sub_tabela_cod_mae_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['cod_mae'] == 11]
df_temporaria3 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'cod_pai']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
sub_tabela_cod_mae_0 = df_temporaria3
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop_duplicates()
sub_tabela_cod_mae_0

# Adicionando codprod, cod_pai, cod_mae no dataframe df_dados_novos_para_insercao_no_banco_de_dados a partir do dataframe sub_tabela_gs_0

In [10]:
df_temporaria4 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae' 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria2
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,anopc,anop_menos1c,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2020,2019,7461,2,2,74612019122,NaN,5,F,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2022,2021,10639,2,2,106392021122,NaN,4,F,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2021,2020,6039,2,2,60392020122,NaN,4,F,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2021,2020,1945,2,2,19452021222,NaN,4,F,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2021,2020,73119,2,2,731192021222,NaN,4,F,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2021,2020,6029,2,2,60292021222,NaN,2,F,NaN
1039,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2019,2018,132,2,2,1322019222,NaN,6,F,NaN
1040,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2021,2020,6915,2,2,69152021222,NaN,5,F,NaN
1041,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2019,2018,6029,2,2,60292019122,NaN,5,F,NaN


# Adicionando cod_pai no df_dados_novos oriundos da subtabela sub_tabela_cod_pai_0

In [ ]:
df_temporaria5 = df_dados_novos_para_insercao_no_banco_de_dados.merge(sub_tabela_cod_pai_0[['ID_ANIMAL', 'RGVACA', 'cod_pai']], on=['ID_ANIMAL', 'RGVACA'], how='left')
df_temporaria5


# Adicionando cod_mae no df_dados_novos oriundos da subtabela sub_tabela_cod_mae_0

In [ ]:
df_temporaria6 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGVACA' 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria2
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados

# Adicionando reb no dataframe df_dados_novos_para_insercao_no_banco_de_dados
Obs: Cod_gil esta na tabela de criadores

In [11]:
df_criadores = df_criadores.rename(columns={'codgiro':'reb_giro'})

df_dados_novos_para_insercao_no_banco_de_dados = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, df_criadores[['reb_giro', 'reb']], on=['reb_giro'], how='left')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,anop_menos1c,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2019,7461,2,2,74612019122,NaN,5,F,NaN,1291
2,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2021,10639,2,2,106392021122,NaN,4,F,NaN,10639-GIRO
3,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2020,6039,2,2,60392020122,NaN,4,F,NaN,NaN
4,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2020,1945,2,2,19452021222,NaN,4,F,NaN,37634-ABCZ
5,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2020,73119,2,2,731192021222,NaN,4,F,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2020,6029,2,2,60292021222,NaN,2,F,NaN,31365-ABCZ
1151,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2018,132,2,2,1322019222,NaN,6,F,NaN,31835-ABCZ
1152,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2020,6915,2,2,69152021222,NaN,5,F,NaN,27729-ABCZ
1153,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2018,6029,2,2,60292019122,NaN,5,F,NaN,31365-ABCZ


In [12]:
#verifica_reb = df_criadores.groupby('reb_giro')['reb'].nunique()
#verifica_reb

## Calculando a diferença entre a data dos novos partos com as ultimas datas presentes na tabela de dados_2022 para validar se ha alguma inconsistência

obs: Caso a diferença seja menor que 250 dias é provável que haja alguma falha.

In [13]:

df_diferenca_entre_partos_novos_e_do_banco = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on=['IDANIMAL'], suffixes=('', '_drop'))
df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop_duplicates()
#df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop(df_diferenca_entre_partos_novos_e_do_banco.filter(regex='_drop$').columns, axis=1)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].astype(str)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].apply(pd.to_datetime)
df_diferenca_entre_partos_novos_e_do_banco['diferenca_em_dias'] = (df_diferenca_entre_partos_novos_e_do_banco['parto'] - df_diferenca_entre_partos_novos_e_do_banco['parto_drop']).dt.days
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop', 'diferenca_em_dias']]

,parto,parto_drop,diferenca_em_dias
0,2021-11-23,2020-01-09,684
1,2020-10-09,2019-01-31,617
2,2021-03-15,2020-01-24,416
3,2021-03-15,2018-02-01,1138
4,2021-06-28,2020-07-22,341
...,...,...,...
682,2021-03-23,2017-11-17,1222
683,2021-03-23,2020-03-10,378
684,2021-01-25,2019-05-03,633
685,2021-04-22,2020-05-13,344


## Pegando da tabela de animais que serão inseridos no banco aqueles que ja possuem IDANIMAL cadastrado
Comparado ao dataframe  dados_2022

In [14]:
df_animais_com_ID = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on =['IDANIMAL'], suffixes=('', '_drop'))
df_animais_com_ID = df_animais_com_ID.drop(df_animais_com_ID.filter(regex='_drop$').columns, axis=1)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,anop_menos1c,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,2020,6029,2,2,60292021122,NaN,2,F,NaN,31365-ABCZ
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,2019,7450,3,2,74502020132,NaN,6,F,NaN,7450-GIRO
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,2020,3304,2,2,33042020122,NaN,4,F,NaN,3304-GIRO
4,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,2020,5569,2,2,55692021222,NaN,4,F,NaN,5569-GIRO
5,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,2020,5569,2,2,55692021222,NaN,3,F,NaN,5569-GIRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,1129,MORADIA ANGICO,7,9556-F,VALE OURO DE BRASILIA,11,A6796,CABANA ANGICO,78,1606,...,2020,1129,2,2,11292021222,NaN,4,F,NaN,52721
681,2381,CLASSICA OCIDENTE JM NOVO HORI,14,5193-AE,OCIDENTE BEM FEITOR CAL,11,CAL4605,CINDERELA MODELO JM NOVO HORIZ,12,AF-2652,...,2020,2381,2,2,23812020122,NaN,2,F,NaN,31300-ABCZ
684,5569,GABI FIV VALE OURO EVD,14,4417-AT,VALE OURO TE SILVANIA,11,EFC464,LAMBADA DRAMATIC SANTA LUZIA,12,AG-0944,...,2020,5569,2,2,55692020122,NaN,2,F,NaN,5569-GIRO
685,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2020,6029,2,2,60292021222,NaN,2,F,NaN,31365-ABCZ


## Pegando da tabela de animais que serão inseridos no banco, todos aqueles que não possuem IDANIMAL cadastrado
comparado ao dataframe dados_2022

In [15]:
df_animais_sem_id = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on='IDANIMAL', how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['_merge'] == 'left_only'].drop(columns='_merge')
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,anop_menos1c,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2019,7461,2,2,74612019122,NaN,5,F,NaN,1291
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2021,10639,2,2,106392021122,NaN,4,F,NaN,10639-GIRO
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2020,6039,2,2,60392020122,NaN,4,F,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2020,1945,2,2,19452021222,NaN,4,F,NaN,37634-ABCZ
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2020,73119,2,2,731192021222,NaN,4,F,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2021,1945,2,2,19452022222,NaN,4,F,NaN,37634-ABCZ
1302,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2020,2381,2,2,23812021222,NaN,4,F,NaN,31300-ABCZ
1305,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2018,132,2,2,1322019222,NaN,6,F,NaN,31835-ABCZ
1306,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2020,6915,2,2,69152021222,NaN,5,F,NaN,27729-ABCZ


In [16]:
#dados_2022[dados_2022['IDANIMAL'] == 1631099]
#df_verificando_animais_com_id_na_genealogia_embrapa['IDANIMAL'].value_counts()
#verifica_reb = df_verificando_animais_com_id_na_genealogia_embrapa.groupby('IDANIMAL')['id_produto'].nunique()
#verifica_reb

# Adicionando a coluna id_produto ao dataframe df_animais_com_ID

In [17]:
df_genealogia_embrapa['IDANIMAL'] = df_genealogia_embrapa['IDANIMAL'].astype('int')
df_animais_com_ID = pd.merge(df_animais_com_ID, df_genealogia_embrapa[['IDANIMAL', 'id_produto']], on='IDANIMAL', how='left', indicator=True)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID = df_animais_com_ID.drop(columns='_merge')
df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb,id_produto
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,6029,2,2,60292021122,NaN,2,F,NaN,31365-ABCZ,GIRO1383626
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,7450,3,2,74502020132,NaN,6,F,NaN,7450-GIRO,GIRO1608903
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,3304,2,2,33042020122,NaN,4,F,NaN,3304-GIRO,GIRO1215331
3,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,5569,2,2,55692021222,NaN,4,F,NaN,5569-GIRO,GIRO2022656
4,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,5569,2,2,55692021222,NaN,3,F,NaN,5569-GIRO,GIRO1653303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,1129,MORADIA ANGICO,7,9556-F,VALE OURO DE BRASILIA,11,A6796,CABANA ANGICO,78,1606,...,1129,2,2,11292021222,NaN,4,F,NaN,52721,GIRO1066128
416,2381,CLASSICA OCIDENTE JM NOVO HORI,14,5193-AE,OCIDENTE BEM FEITOR CAL,11,CAL4605,CINDERELA MODELO JM NOVO HORIZ,12,AF-2652,...,2381,2,2,23812020122,NaN,2,F,NaN,31300-ABCZ,GIRO1571455
417,5569,GABI FIV VALE OURO EVD,14,4417-AT,VALE OURO TE SILVANIA,11,EFC464,LAMBADA DRAMATIC SANTA LUZIA,12,AG-0944,...,5569,2,2,55692020122,NaN,2,F,NaN,5569-GIRO,GIRO1797713
418,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,6029,2,2,60292021222,NaN,2,F,NaN,31365-ABCZ,GIRO1383629


## Lista de animais do dataframe sem ID em relacao a dados_2022 e que possuem ID_animal no dataframe da genealogia_Embrapa
Obs: Tem que ter ID_produto

In [18]:
df_animais_sem_id = df_animais_sem_id.add_suffix('_esq')
df_genealogia_embrapa = df_genealogia_embrapa.add_suffix('_dir')

df_animais_sem_id_2022_que_possuem_id_no_banco = pd.merge(df_animais_sem_id, df_genealogia_embrapa, left_on=['IDANIMAL_esq'], right_on=['IDANIMAL_dir'], how='left', indicator=True)
df_animais_sem_id_2022_que_possuem_id_no_banco.drop_duplicates()
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco[df_animais_sem_id_2022_que_possuem_id_no_banco['_merge'] == 'both']

# dropa as colunas que não são necessárias
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_dir').columns.difference(['id_produto_dir']), axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
df_animais_sem_id_2022_que_possuem_id_no_banco

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb,id_produto
9,1945,JPZ RENDEIRA TEATRO GENOVA FIV,12,7881-AD,TEATRO DA SILVANIA,11,EFC383,PIONEIRA DO SUL GENOVA BARCELO,10,BX338026,...,1945,2,2,19452021222,NaN,4,F,NaN,37634-ABCZ,GIRO1548230
21,7461,PORCINA RF SAO FRANCISCO,38,4082-AX,EURO TE JABAQUARA,11,EVPF425,LORADA LAVEE RF SAO FRANCISCO,34,7500-V,...,7461,2,2,74612020122,NaN,3,F,NaN,1291,216013
25,9000,ICH Q379 CANELA TABU,14,9488-AN,TABU TE DA CAL,11,CAL6557,ICH CANELA TEATRO,12,S-3377,...,9000,2,2,90002020222,NaN,2,F,NaN,NaN,GIRO1853043
45,2915,BOLIVIA SHREK 2R JATAI,12,3784-M,J.D.F. SHREK 932,10,AX115283,HEMERA FIV STAR,11,FRFL273,...,2915,1,2,29152020112,NaN,4,F,NaN,2915-GIRO,GIRO1245256
46,8812,BATALHA VALE OURO SERRA DO LUA,12,3200-J,VALE OURO TE SILVANIA,11,EFC464,V.D.V. GUIRACA GILBERT 692,10,BX370510,...,8812,2,2,88122020122,NaN,4,F,NaN,8812-GIRO,GIRO1267672
58,7461,IRONDINA ROGER RF SAO FRANCISC,58,6886-N,FREUREHAVEN ROGER,10,AX125592,EDUARDA BARBANTE RF SAO FRANCI,38,C-8737,...,7461,2,2,74612020122,NaN,5,F,NaN,1291,GIRO1333698
70,7450,JEITOSA FIV DELEGADO SJ LALU,12,6403-AH,DELEGADO,11,BJAS204,GERALDO ANITA SUEDE 829,10,BX375920,...,7450,2,2,74502018122,NaN,4,F,NaN,7450-GIRO,GIRO1593521
98,7461,ELITE RF SAO FRANCISCO,78,6715-N,0,0,0,0,0,0,...,7461,2,2,74612020122,NaN,7,F,NaN,1291,GIRO1346102
136,5426,HEGLA BELLA ESPELHO FIV DA ZEN,12,1080-AM,ESPELHO TE DE BRASILIA,11,RRP5664,A.C.B. ADVENT RED BELLA,10,BX401454,...,5426,2,2,54262019122,NaN,4,F,NaN,34101-ABCZ,GIRO1752044
161,4237,CARLA EXPOENTE FUNDAO,14,7344-U,EXPOENTE DE BRASILIA,11,B-0208,SUZY LEDUC FUNDAO,12,AF-3243,...,4237,2,2,42372021222,NaN,2,F,NaN,4394-ABCZ,GIRO1367750


# Criando dataframe com todos os animais que possuem ID encontrado

In [19]:
# Juntando os dataframes
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_sem_id_2022_que_possuem_id_no_banco])
df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb,id_produto
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,6029,2,2,60292021122,NaN,2,F,NaN,31365-ABCZ,GIRO1383626
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,7450,3,2,74502020132,NaN,6,F,NaN,7450-GIRO,GIRO1608903
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,3304,2,2,33042020122,NaN,4,F,NaN,3304-GIRO,GIRO1215331
3,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,5569,2,2,55692021222,NaN,4,F,NaN,5569-GIRO,GIRO2022656
4,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,5569,2,2,55692021222,NaN,3,F,NaN,5569-GIRO,GIRO1653303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,8689,JOA EMBAIXADOR LAGLORIA,14,6341-M,EMBAIXADOR LAGLORIA,11,MAFV20,FLORINDA LAGLORIA,12,AE-2426,...,8689,2,2,86892019122,NaN,2,F,NaN,8689-GIRO,GIRO1161387
544,9373,JAB BARONEZA OPINION,78,8900-AI,ROYAL OPINION DECEMBER,10,AX119971,INTEGRALAT FIV 77761 KIRKLAND,34,AP-1389,...,9373,2,2,93732020122,NaN,7,F,NaN,52706,GIRO1518407
563,5192,SUGESTIVA FIV DE BRAS.,11,RRP8637,JAGUAR TE DO GAVIAO,11,GAV291,JESSICA DE BRAS.,11,RRP6775,...,5192,3,2,51922020132,NaN,0,F,16309606.0,5192-GIRO,RRP8637
569,6029,GRUTA FR RECREIO,12,4847-BG,0,0,0,0,0,0,...,6029,2,2,60292021222,NaN,4,F,NaN,31365-ABCZ,214852


# Atualizando lista de animais sem ID

In [20]:
#Tirando o sufixo da tabela
df_animais_sem_id = df_animais_sem_id.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
# Imprimir o resultado
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto'].isnull()]
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb,id_produto,_merge
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,2,74612019122,NaN,5,F,NaN,1291,NaN,left_only
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2,106392021122,NaN,4,F,NaN,10639-GIRO,NaN,left_only
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,2,60392020122,NaN,4,F,NaN,NaN,NaN,left_only
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,2,19452021222,NaN,4,F,NaN,37634-ABCZ,NaN,left_only
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,2,731192021222,NaN,4,F,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2,2,19452022222,NaN,4,F,NaN,37634-ABCZ,NaN,left_only
620,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2,2,23812021222,NaN,4,F,NaN,31300-ABCZ,NaN,left_only
621,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,2,1322019222,NaN,6,F,NaN,31835-ABCZ,NaN,left_only
622,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,2,69152021222,NaN,5,F,NaN,27729-ABCZ,NaN,left_only


# Fazendo busca no dataframe producao para encontrar id_produto dos animais sem id
conferir manualmente se entram ou não no banco

In [21]:
#Criando coluna com o ano de parto e mes parto no dataframe df_producao
df_producao['parto'] = df_producao['parto'].astype(str)
df_producao['AP'] = df_producao['parto'].str.slice(start=0, stop=4)
df_producao['MP'] = df_producao['parto'].str.slice(start=5, stop=7)
#Pegando os 10 primeiros caracteres de nome dos dataframes aniamis sem id e producao
df_producao['nome10caracteres'] = df_producao['nome'].str.slice(start=0, stop=11)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nomea'].str.slice(start=0, stop=11)
#removendo os espacos em branco das celulas
df_producao['nome10caracteres'] = df_producao['nome10caracteres'].str.replace('\s+', '', regex=True)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nome10caracteres'].str.replace('\s+', '', regex=True)

df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_producao = df_producao.drop(df_producao.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns='_merge')

df_producao['AP'] = df_producao['AP'].astype(int)
df_producao['MP'] = df_producao['MP'].astype(int)
df_animais_sem_id['AP'] = df_animais_sem_id['AP'].astype(int)
df_animais_sem_id['MP'] = df_animais_sem_id['MP'].astype(int)
df_animais_com_id_na_producao = pd.merge(df_animais_sem_id, df_producao[['reb','AP', 'MP', 'nome10caracteres','id_produto']], on=['reb', 'AP', 'MP', 'nome10caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_producao = df_animais_com_id_na_producao[df_animais_com_id_na_producao['_merge'] == 'both']
df_animais_com_id_na_producao = df_animais_com_id_na_producao.drop(columns={'_merge', 'id_produto'})
df_animais_com_id_na_producao = df_animais_com_id_na_producao.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_producao

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb,nome10caracteres,id_produto
216,11818,BACANA CABO VERDE,11,JCVL2511,GABEIRA GIROESTE,11,LUF182,VISCAYA CABO VERDE,11,JCVL1482,...,3,2,118182021232,NaN,0,F,16530248.0,39977-ABCZ,BACANACABO,JCVL2511
388,2381,GALENA FABULOSO JM NOVO HORIZO,38,7020-BC,FABULOSO DO BASA,11,BASP63,ALVORADA 122 JM NOVO HORIZONTE,34,2686-AG,...,2,2,23812021222,NaN,3,F,NaN,31300-ABCZ,GALENAFABU,216096
459,2381,FANTASIA FABULOSO JM NOVO HORI,38,4896-AZ,FABULOSO DO BASA,11,BASP63,NOVIAA I FIV GOLDEN JM NOVO HO,34,3673-U,...,2,2,23812021222,NaN,3,F,NaN,31300-ABCZ,FANTASIAFA,216094


# Atualizando animais com id com os novos ids_produtos encontrados no dataframe de producao

In [22]:
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_com_id_na_producao])
df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb,id_produto,nome10caracteres
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,2,2,60292021122,NaN,2,F,NaN,31365-ABCZ,GIRO1383626,NaN
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,3,2,74502020132,NaN,6,F,NaN,7450-GIRO,GIRO1608903,NaN
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,2,2,33042020122,NaN,4,F,NaN,3304-GIRO,GIRO1215331,NaN
3,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,2,2,55692021222,NaN,4,F,NaN,5569-GIRO,GIRO2022656,NaN
4,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,2,2,55692021222,NaN,3,F,NaN,5569-GIRO,GIRO1653303,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,6029,GRUTA FR RECREIO,12,4847-BG,0,0,0,0,0,0,...,2,2,60292021222,NaN,4,F,NaN,31365-ABCZ,214852,NaN
591,1945,ABAMA 2 FIV M. VERDE,11,ISPG1286,C.A.SANSAO,11,KCA472,ABAMA TE DE KUBERA,11,ACFG80,...,2,2,19452021222,NaN,0,F,11869706.0,37634-ABCZ,ISPG1286,NaN
216,11818,BACANA CABO VERDE,11,JCVL2511,GABEIRA GIROESTE,11,LUF182,VISCAYA CABO VERDE,11,JCVL1482,...,3,2,118182021232,NaN,0,F,16530248.0,39977-ABCZ,JCVL2511,BACANACABO
388,2381,GALENA FABULOSO JM NOVO HORIZO,38,7020-BC,FABULOSO DO BASA,11,BASP63,ALVORADA 122 JM NOVO HORIZONTE,34,2686-AG,...,2,2,23812021222,NaN,3,F,NaN,31300-ABCZ,216096,GALENAFABU


# Atualizando lista de animais sem id

In [23]:
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto_drop'].isnull()]
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,reb,id_produto,nome10caracteres,_merge
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,74612019122,NaN,5,F,NaN,1291,NaN,NANAIMPERA,left_only
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,106392021122,NaN,4,F,NaN,10639-GIRO,NaN,LANAAFIV,left_only
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,60392020122,NaN,4,F,NaN,NaN,NaN,EDILAINEIM,left_only
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,19452021222,NaN,4,F,NaN,37634-ABCZ,NaN,ARMAMERIDI,left_only
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,731192021222,NaN,4,F,NaN,NaN,NaN,CELESTE136,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2,19452022222,NaN,4,F,NaN,37634-ABCZ,NaN,JPZQUIRERA,left_only
579,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2,23812021222,NaN,4,F,NaN,31300-ABCZ,NaN,DALIAIFIV,left_only
580,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,1322019222,NaN,6,F,NaN,31835-ABCZ,NaN,QUINTAICO,left_only
581,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,69152021222,NaN,5,F,NaN,27729-ABCZ,NaN,LIADASARA,left_only


# Fazendo a busca na tabela genealogia_embrapa

In [24]:
#Retirando sufixo tabela genealogia
df_genealogia_embrapa = df_genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
#Pegando ano nascimento nas colunas
df_genealogia_embrapa['nasc_est'] = df_genealogia_embrapa['nasc_est'].astype(str)
df_genealogia_embrapa['AN'] = df_genealogia_embrapa['nasc_est'].str.slice(start=0, stop=4)
df_animais_sem_id['nasc'] = df_animais_sem_id['nasc'].astype(str)
df_animais_sem_id['AN'] = df_animais_sem_id['nasc'].str.slice(start=0, stop=4)
#Criando coluna com os 10 caracteres do nome na genealogia_embrapa
df_genealogia_embrapa['nome10caracteres'] = df_genealogia_embrapa['nome'].str.slice(start=0, stop=11)
#Convertendo coluna ano e mes para tipo inteiro
#df_genealogia_embrapa['AN'] = df_genealogia_embrapa['AN'].astype(int)
#Limpando tabelas
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_genealogia_embrapa = df_genealogia_embrapa.drop(df_genealogia_embrapa.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns={'_merge'})
#criando dataframe com os ids_encontrados
df_animais_com_id_na_genealogia = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs','RGMAE', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia[df_animais_com_id_na_genealogia['_merge'] == 'both']
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(columns={'_merge', 'id_produto', 'registro_mae', 'registro_pai'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(df_animais_com_id_na_genealogia.filter(regex='_drop$').columns, axis=1)
df_animais_com_id_na_genealogia

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,ordem,gs,sexo,cod_prod,reb,nome10caracteres,AN,registro_mae,registro_pai,id_produto


In [ ]:
#Conferência manual dos matchs com o dataframe da genealogia

In [26]:
df_animais_com_id_na_genealogia1 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia1 = df_animais_com_id_na_genealogia1[df_animais_com_id_na_genealogia1['_merge'] == 'both']
df_animais_com_id_na_genealogia1

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,gs,sexo,cod_prod,reb,id_produto,nome10caracteres,AN,registro_pai,id_produto_drop,_merge
414,1945,FANTASIA,11,MBLO18,JAGUAR TE DO GAVIAO,11,GAV291,ELECTRA FIV PEDREIRA,11,PPG41,...,0,F,17697290.0,37634-ABCZ,NaN,FANTASIA,2020,GAV291,TAN219,both


# Adicionar o que ta escrito no quadro

In [27]:
df_animais_com_id_na_genealogia1 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia1 = df_animais_com_id_na_genealogia1[df_animais_com_id_na_genealogia1['_merge'] == 'both']
df_animais_com_id_na_genealogia1

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,ordem,gs,sexo,cod_prod,reb,id_produto,nome10caracteres,AN,id_produto_drop,_merge
409,1945,BAILARINA,11,MBLO12,GENGIS KHAN DE BRASILIA,11,RRP6097,BEL DA RTPA,11,NLT89,...,NaN,0,F,17695912.0,37634-ABCZ,NaN,BAILARINA,2020,RGIR34,both
414,1945,FANTASIA,11,MBLO18,JAGUAR TE DO GAVIAO,11,GAV291,ELECTRA FIV PEDREIRA,11,PPG41,...,NaN,0,F,17697290.0,37634-ABCZ,NaN,FANTASIA,2020,TAN219,both


In [30]:
df_animais_com_id_na_genealogia2 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['id_produto','cod_prod']], left_on=['cod_prod'], right_on=['cod_prod'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia2 = df_animais_com_id_na_genealogia2[df_animais_com_id_na_genealogia2['_merge'] == 'both']
df_animais_com_id_na_genealogia2

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,ordem,gs,sexo,cod_prod,reb,id_produto,nome10caracteres,AN,id_produto_drop,_merge
104,5192,URUPIANGA FIV DE BRAS.,11,RRP8772,JAGUAR TE DO GAVIAO,11,GAV291,HONDA FIV DE BRAS.,11,RRP6268,...,NaN,0,F,17054525.0,5192-GIRO,NaN,URUPIANGAF,2019,RRP8772,both
129,5192,PRENDA FIV DE BRAS.,11,RRP8191,GENGIS KHAN DE BRASILIA,11,RRP6097,FERVURA FIV DE BRASILIA,11,RRP5998,...,NaN,0,F,14926361.0,5192-GIRO,NaN,PRENDAFIV,2015,RRP8191,both
184,22541,FELICE FIV CABO VERDE,11,JCVL3334,JAGUAR TE DO GAVIAO,11,GAV291,ZILAH FIV CABO VERDE,11,JCVL1624,...,NaN,0,F,17198554.0,NaN,NaN,FELICEFIV,2019,JCVL3334,both
223,42583,BARILOCHE KENYO,11,KOK844,ELDORADO FIV KENYO,11,KOK96,IZABELLE FIV KENYO,11,KOK526,...,NaN,0,F,14672175.0,NaN,NaN,BARILOCHEK,2014,KOK844,both
269,5192,RAGIN FIV DE BRAS.,11,RRP8395,JAGUAR TE DO GAVIAO,11,GAV291,ISCA DE BRASILIA,11,RRP6699,...,NaN,0,F,15421321.0,5192-GIRO,NaN,RAGINFIVD,2016,RRP8395,both
292,5192,RUANA DE BRAS.,11,RRP8430,BACO DA RTPA,11,NLT130,LALA DE BRAS.,11,RRP7119,...,NaN,0,F,15438291.0,5192-GIRO,NaN,RUANADEBR,2016,RRP8430,both
304,5192,LIZ TEATRO,11,RARS10,TEATRO DA SILVANIA,11,EFC383,RENUNCIA FIV DE BRAS.,11,RRP8249,...,NaN,0,F,17112475.0,5192-GIRO,NaN,LIZTEATRO,2019,RARS10,both
326,42583,MILENA KENYO,11,KOK846,NOBRE TE CAL,11,CAL4397,JARANA FIV KENYO,11,KOK625,...,NaN,0,F,14673091.0,NaN,NaN,MILENAKENY,2015,KOK846,both
442,5192,REDE FIV DE BRAS.,11,RRP8488,GENGIS KHAN DE BRASILIA,11,RRP6097,DELI DE BRAS.,11,RRP5562,...,NaN,0,F,15617034.0,5192-GIRO,NaN,REDEFIVDE,2016,RRP8488,both
453,5192,RENUNCIA FIV DE BRAS.,11,RRP8249,FARDO FIV F.MUTUM,11,MUT697,ESFERA DE BRASILIA TN2 DE BRAS,11,RRP7530,...,NaN,0,F,15128061.0,5192-GIRO,NaN,RENUNCIAFI,2016,RRP8249,both
